In [ ]:
!pip install yfinance
!pip install scalecast
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
import seaborn as sns
from random import random
from scipy.stats import norm
from scipy.stats import stats
from scalecast.Forecaster import Forecaster

# Rolling Sharpe

In [ ]:

def get_returns(ticker, start_date):
    # Pull Historical Data
    data = yf.download(ticker, start=start_date)
    # Calculate Daily Returns
    data['Daily Return'] = data['Adj Close'].pct_change()
    return data.dropna()

#Rolling sharpe calculation (daily return pct divided by std of returns)
tnx = yf.Ticker('^TNX')
risk_free_rate = tnx.info.get('previousClose') * 0.01

data = get_returns('ON', '2022-08-01')
data['sharpe'] = (data['Daily Return'] - risk_free_rate) / np.std(data['Daily Return'])
data['sharpe'].plot(kind='line', figsize=(15,6))


# Discounted Cash Flow Analysis

In [ ]:
class DCF:

  def __init__(self, symbol, proj_growth, tgr):
    self._ticker = yf.Ticker(symbol)
    # projected growth rates for operating cash flow
    self._proj_growth = proj_growth
    # forecasting period
    self._period = len(self._proj_growth)
    # terminal growth rate
    self._tgr = tgr

    self._info = self._ticker.info
    # reverse col order (ascending FYE)
    self._financials = self._ticker.financials[self._ticker.financials.columns[::-1]]
    self._balance_sheet = self._ticker.balance_sheet[self._ticker.balance_sheet.columns[::-1]]
    self._cash_flow = self._ticker.cashflow[self._ticker.cashflow.columns[::-1]]
    self._wacc = self.wacc()
    # round all figures to 2 decimal places
    pd.set_option('display.float_format', lambda x: '%.2f' % x)

  def prep(self):
    op_cf = self._cash_flow.loc['Total Cash From Operating Activities']
    capex = self._cash_flow.loc['Capital Expenditures']
    # CapEx as % of operating cash flow
    pcnts_capex_op_cf = capex.multiply(-1).div(op_cf)
    avg_pcnt = pcnts_capex_op_cf.mean()
    df_prep = self._cash_flow.loc[['Total Cash From Operating Activities']]
    df_prep.loc['CapEx'] = capex.multiply(-1)
    for i in range(self._period):
      proj_op_cf = df_prep.iat[0, -1] * (1 + self._proj_growth[i])
      proj_capex = avg_pcnt * proj_op_cf
      df_prep[i + 1] = [proj_op_cf, proj_capex]
    df_prep.loc['Free Cash Flow'] = df_prep.sum()
    return df_prep

  def dcf(self):
    df_dcf = self.prep()
    wacc = self._wacc
    df_dcf.loc['Present Value of FCF'] = 0
    for i in range(self._period):
      curr_fcf = df_dcf.at['Free Cash Flow', i + 1] 
      df_dcf.at['Present Value of FCF', i + 1] = curr_fcf / (1 + wacc)**(i + 1)
    return df_dcf

  def share_price(self):
    df_dcf = self.dcf()
    wacc = self._wacc
    last_fcf = df_dcf.at['Free Cash Flow', self._period]
    tv = (last_fcf * (1 + self._tgr)) / (wacc - self._tgr)
    pv_tv = tv / (1 + wacc)**(self._period)
    enterprise_value = df_dcf.loc['Present Value of FCF'].sum() + pv_tv
    cash = self._balance_sheet.loc['Cash'][-1]
    debt = self._balance_sheet.loc['Long Term Debt'][-1]
    equity_value = enterprise_value + cash - debt
    shares = self._info.get('sharesOutstanding')
    share_price = equity_value / shares
    d = [tv, pv_tv, enterprise_value, cash, debt, equity_value, shares, 
         share_price]
    df_sp = pd.DataFrame(data=d, columns=['All numbers in dollars'])
    df_sp.index = ['Terminal Value', 'Present Value of Terminal Value',
                   'Enterprise Value', 'Cash', 'Debt', 'Equity Value', 'Shares',
                   'Implied Share Price']
    return df_sp

  def wacc(self):
    # treasury yield for risk free rate
    tnx = yf.Ticker('^TNX')
    rfr = tnx.info.get('previousClose') * 0.01
    # beta
    b = self._info.get('beta')
    # equity risk premium
    erp = 0.056

    # calculate mean tax rate
    taxes = self._financials.loc['Income Tax Expense'].abs()
    ebit = self._financials.loc['Ebit']
    tax_rates = taxes.div(ebit)
    tc = tax_rates.mean()
   
    # calculate cost of equity
    cost_equity = rfr + b * (erp - rfr)
    # market value of equity (market capitalization)
    e = self._info.get('marketCap')

    # calculate cost of debt
    interests = self._financials.loc['Interest Expense'].multiply(-1)
    debts = self._balance_sheet.loc['Long Term Debt']
    int_rates = interests.div(debts)
    avg_int_rate = int_rates.mean()
    cost_debt = avg_int_rate * (1 - tc)
    # market value of debt (most recent debt figure)
    d = debts[-1]

    # for ratios
    v = e + d

    # equation
    wacc = (e/v * cost_equity) + (d/v * cost_debt * (1 - tc))
    return wacc

# DCF Example: $NVDA

In [ ]:
# symbol = 'NVDA'
# proj_growth = [0.35, 0.25, 0.15]
# tgr = 0.025
# nvda = DCF(symbol, proj_growth, tgr)

In [ ]:
# nvda.wacc()

WACC = (E/V * Re) + (D/V * Rd * (1 - Tc))


WACC = Weighted Average Cost of Capital

E = Market Value of Equity

D = Market Value of Debt

V = E + D

Re = Cost of Equity

Rd = Cost of Debt

Tc = Corporate Tax Rate

In [ ]:
# nvda.prep()

Free Cash Flow (FCF) = Operating Cash Flow - Capital Expenditures

In [ ]:
# nvda.dcf()

Present Value of FCF = FCF / (1 + WACC)^n

Current Share Price: $125.99

In [ ]:
# nvda.share_price()

Terminal Value = (FCF_k * (1 + TGR)) / (WACC - TGR)

Present Value of Terminal Value = Terminal Value / (1 + WACC)^k

Enterprise Value = (sum of all Present Value of FCF) + Present Value of Terminal Value

Equity Value = Enterprise Value + Cash - Debt

Implied Share Price = Equity Value / Shares

# DCF: $GFL

In [ ]:
symbol = 'ON'
proj_growth = [0.15, 0.10, 0.05] # operating cash flow
tgr = 0.025 # terminal growth rate
gfl = DCF(symbol, proj_growth, tgr)

In [ ]:
gfl.wacc()

In [ ]:
gfl.prep()

In [ ]:
gfl.dcf()

Current Share Price: $25.64

In [ ]:
gfl.share_price()

# Monte Carlo Model

In [ ]:
# Defining the Ticker
ticker = yf.Ticker('ON')

# Obtaining Historical Market Data
start_date = '2020-05-01'
end_date = '2022-11-04'
hist = ticker.history(start=start_date, end=end_date)
#print(hist.head())

# Pulling Closing Price Data
hist = hist[['Close']]
print(hist)

# Plotting Price Data
hist['Close'].plot(title="GFL Stock Price", ylabel=
                   "Closing Price [$]", figsize=[10, 6])
plt.grid()

In [ ]:
# Create Day Count, Price, and Change Lists
days = [i for i in range(1, len(hist['Close'])+1)]
price_orig = hist['Close'].tolist()
change = hist['Close'].pct_change().tolist()
change = change[1:]  # Removing the first term since it is NaN

# Statistics for Use in Model
mean = np.mean(change)
std_dev = np.std(change)
print('\nMean percent change: ' + str(round(mean*100, 2)) + '%')
print('Standard Deviation of percent change: ' +   
      str(round(std_dev*100, 2)) + '%')

In [ ]:
# Simulation Number and Prediction Period
simulations = 100 # Change for more results
days_to_sim = 1*252 # Trading days in 1 year

# Initializing Figure for Simulation
fig = plt.figure(figsize=[10, 6])
plt.plot(days, price_orig)
plt.title("Monte Carlo Stock Prices [" + str(simulations) + 
          " simulations]")
plt.xlabel("Trading Days After " + start_date)
plt.ylabel("Closing Price [$]")
plt.xlim([0, len(days)+days_to_sim])
plt.grid()

# Initializing Lists for Analysis
close_end = []
above_close = []
hpr = [] # holding period returns

# For Loop for Number of Simulations Desired
for i in range(simulations):
    num_days = [days[-1]]
    close_price = [hist.iloc[-1, 0]]
    last_price = close_price[-1] # before sim

    # For Loop for Number of Days to Predict
    for j in range(days_to_sim):
        num_days.append(num_days[-1]+1)
        perc_change = norm.ppf(random(), loc=mean, scale=std_dev)
        close_price.append(close_price[-1]*(1+perc_change)) # random walk

    if close_price[-1] > price_orig[-1]:
        above_close.append(1)
    else:
        above_close.append(0)

    close_end.append(close_price[-1])
    
    # calculate holding period return
    curr_hpr = (close_end[-1] - last_price)/last_price * 100
    hpr.append(curr_hpr)

    plt.plot(num_days, close_price)

# Average Closing Price and Probability of Increasing After 1 Year
average_closing_price = sum(close_end)/simulations
average_perc_change = (average_closing_price-
                       price_orig[-1])/price_orig[-1]
probability_of_increase = sum(above_close)/simulations
print('\nPredicted closing price after ' + str(simulations) + 
      ' simulations: $' + str(round(average_closing_price, 2)))
print('Predicted percent increase after 1 year: ' + 
      str(round(average_perc_change*100, 2)) + '%')
print('Probability of stock price increasing after 1 year: ' + 
      str(round(probability_of_increase*100, 2)) + '%')

# Displaying the Monte Carlo Simulation Lines
plt.show()

In [ ]:
sns.histplot(hpr, kde=True, bins=20, color='green')
plt.title('Holding Period Return Distribution')
plt.xlabel('HPR %')
plt.axvline(np.percentile(hpr, 5), color='r', linestyle='dashed',
            linewidth=2)
plt.axvline(np.percentile(hpr, 95), color='r', linestyle='dashed',
            linewidth=2)
plt.show()
# Skewed right

# Time Series Forecasting: LSTM

In [ ]:
gfl = yf.Ticker('ON')
close = gfl.history(period='max')[['Close']]
close.index = close.index.date
close

In [ ]:
close.plot(figsize=(15, 6))

In [ ]:
f = Forecaster(y=close['Close'], current_dates=close.index)
f.plot_pacf(lags=34)

In [ ]:
f.set_test_length(800) # Testing using ~ 30% of data
f.generate_future_dates(183) # 1 year forecast
f.set_estimator('lstm')
f.manual_forecast(call_me='lstm_default')
f.plot_test_set(ci=True)

In [ ]:
f.manual_forecast(call_me='predictions',
                  lags=10,
                  epochs=12,
                  validation_split=.2,
                  shuffle=True)
f.plot_test_set(order_by='LevelTestSetMAPE', models='top_2', ci=True)

In [ ]:
f.plot(models=['predictions'],
       order_by='LevelTestSetMAPE',
       level=True)

In [ ]:
vals = f.export('best_fcst')
del vals['MODEL']
vals.index = vals['DATE'].dt.strftime('%Y-%m-%d')
del vals['DATE']
vals.to_json()